# VideoPose Visualization

### Basic Imports

In [1]:
import matplotlib
matplotlib.use('TKAgg')

import numpy as np
import matplotlib.pyplot as plt
from common.visualization import render_animation
from common.generators import ChunkedGenerator, UnchunkedGenerator
from common.camera import *
from common.model import *
from common.loss import *
from common.skeleton import Skeleton
import os
import json

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
from mpl_toolkits.mplot3d import Axes3D
import subprocess as sp

In [2]:
def read_json(filename, datapath="/datasets_local/DriveAndAct"):
    try:
        if filename.endswith("json"):
            data = json.load(open(os.path.join(datapath, filename), "r"))
        else:
            data = np.load(os.path.join(datapath, filename), allow_pickle=True)
        return data
    except TypeError:
        " You are trying to load a wrong type of file (Must be JSON or NPZ)"

In [13]:
from matplotlib.pyplot import colorbar


def downsample_tensor(X, factor):
    length = X.shape[0]//factor * factor
    return np.mean(X[:length].reshape(-1, factor, *X.shape[1:]), axis=1)

def render_animation(keypoints_2d, poses_3d, images, skeleton, 
                     output='output.mp4',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=3, size=6, azim=180, bitrate=-1, limit=1000, elev=10):
    """_summary_

    Args:
        keypoints_2d (_type_): _description_
        poses_3d (_type_): _description_
        images (list): list of images names
        skeleton (Skeleton): skeleton attributes
        output (str, optional): _description_. Defaults to 'output.mp4'.
        datapath (str, optional): path to dataset folder. Defaults to '/datasets_local/DriveAndAct'.
        skip (int, optional): _description_. Defaults to 0.
        fps (int, optional): _description_. Defaults to 30.
        downsample (int, optional): decreasing factor for video speed   . Defaults to 3.
        size (int, optional): _description_. Defaults to 6.
        azim (int, optional): _description_. Defaults to 180.

    Raises:
        ValueError: _description_
    """
    plt.ioff()
    fig = plt.figure(figsize=(size*(1 + len(poses_3d)), size))
    ax_in = fig.add_subplot(1, 1 + len(poses_3d), 1)
    ax_in.get_xaxis().set_visible(False)
    ax_in.get_yaxis().set_visible(False)
    ax_in.set_axis_off()
    ax_in.set_title('Input')

    ax_3d = []
    lines_3d = []
    trajectories = []
    all_frames = []
    radius = 1.7
    # for index, (data) in enumerate(poses_3d[0]): #.items()
    ax = fig.add_subplot(1, 1 + len(poses_3d), 2, projection='3d')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlim3d([-radius/2, radius/2])
    ax.set_ylim3d([0, radius])
    ax.set_zlim3d([-radius/2, radius/2])
    try:
        ax.set_aspect('equal')
    except NotImplementedError:
        ax.set_aspect('auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    ax.dist = 7.5
    ax.set_title("Ground truth") #, pad=35
    ax_3d.append(ax)
    lines_3d.append([])
    for index in range(min(limit, len(poses_3d[0]))):
        all_frames.append(plt.imread(os.path.join(datapath, images[index])))
        # trajectories.append(data[:, 0, [0, 1]])
    # poses_3d = list(poses_3d.values())
    poses_3d = list(poses_3d)
    print("frames length: ", np.array(all_frames).shape, len(all_frames), all_frames[0].shape)
  
    keypoints = keypoints_2d
    
    if downsample > 1:
        keypoints = downsample_tensor(keypoints_2d, downsample)
        all_frames = downsample_tensor(np.array(all_frames), downsample).astype('uint8')
        for idx in range(len(poses_3d[0])):
            poses_3d[0][idx] = downsample_tensor(poses_3d[0][idx], downsample)
            # trajectories[idx] = downsample_tensor(trajectories[idx], downsample)
        fps /= downsample
        
        
    initialized = False
    image = None
    lines = []
    points = None
    print()
    
    if limit < 1:
        limit = len(all_frames)
    else:
        limit = min(limit, len(all_frames))
        
    parents = skeleton.parents()
    def update_video(i):
        nonlocal initialized, image, lines, points

        """
        for n, ax in enumerate(ax_3d):
            ax.set_xlim3d([-radius/2 + trajectories[n][i, 0], radius/2 + trajectories[n][i, 0]])
            ax.set_ylim3d([-radius/2 + trajectories[n][i, 1], radius/2 + trajectories[n][i, 1]])
        """
        # Update 2D poses
        joints_right_2d = [2, 4, 6, 8, 10, 12, 14, 16]
        colors_2d = np.full(keypoints.shape[1], 'black')
        colors_2d[joints_right_2d] = 'red'
        if not initialized:
            image = ax_in.imshow(all_frames[i], aspect='equal')
            
            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue
                    
                if len(parents) == keypoints.shape[1]:
                    # Draw skeleton only if keypoints match (otherwise we don't have the parents definition)
                    if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
                        lines.append(ax_in.plot([keypoints[i, j, 0], keypoints[i, j_parent, 0]],
                                                [keypoints[i, j, 1], keypoints[i, j_parent, 1]], color='pink'))
                    else:
                        lines.append(ax_in.plot([0, 0],
                                                [0, 0], color='pink'))
                col = 'red' if j in skeleton.joints_right() else 'black'
                # for n, ax in enumerate(ax_3d):
                n = 0
                pos = poses_3d[n][i]
                if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
                    lines_3d[n].append(ax.plot([pos[j, 0], pos[j_parent, 0]],
                                               [-pos[j, 1], -pos[j_parent, 1]],
                                               [pos[j, 2], pos[j_parent, 2]], zdir='y', color=col))
                else:
                    lines_3d[n].append(ax.plot([0, 0], [0, 0], [0, 0], zdir='y', color="white"))

            if keypoints[i, 11, 2] != 0 and keypoints[i, 12, 2] != 0 :
                col = "blue"
            else:
                col = "white"
            extra_parent = 12
            lines.append(ax_in.plot([keypoints[i, 11, 0], keypoints[i, 12, 0]],
                                            [keypoints[i, 11, 1], keypoints[i, 12, 1]], color=col))
            lines_3d[n].append(ax.plot([pos[11, 0], pos[extra_parent, 0]],
                                        [-pos[11, 1], -pos[extra_parent, 1]],
                                        [pos[11, 2], pos[extra_parent, 2]], zdir='y', color=col))
            if keypoints[i, 5, 2] != 0 and keypoints[i, 6, 2] != 0 :
                col = "blue"
            else:
                col = "white"
            extra_parent = 6
            lines.append(ax_in.plot([keypoints[i, 5, 0], keypoints[i, 6, 0]],
                                            [keypoints[i, 5, 1], keypoints[i, 6, 1]], color=col))
            lines_3d[n].append(ax.plot([pos[5, 0], pos[extra_parent, 0]],
                                        [-pos[5, 1], -pos[extra_parent, 1]],
                                        [pos[5, 2], pos[extra_parent, 2]], zdir='y', color=col))
            
                
                
            
            points = ax_in.scatter(*keypoints[i, :, :2].T, 10, color=colors_2d, edgecolors='white', zorder=10) # replaced colors_2d

            initialized = True

        else:
            
            image.set_data(all_frames[i])

            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue
                
                if len(parents) == keypoints.shape[1]:
                    lines[j-1][0].set_color('pink')
                    if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
                        lines[j-1][0].set_data([keypoints[i, j, 0], keypoints[i, j_parent, 0]],
                                            [keypoints[i, j, 1], keypoints[i, j_parent, 1]])
                        
                    else:
                        lines[j-1][0].set_color('white')
                        lines[j-1][0].set_data([0, 0], [0, 0])
                        

                # for n, ax in enumerate(ax_3d):
                n = 0
                pos = poses_3d[0][i]
                if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
                    lines_3d[n][j-1][0].set_color(colors_2d[j-1])
                    lines_3d[n][j-1][0].set_xdata(np.array([pos[j, 0], pos[j_parent, 0]]))
                    lines_3d[n][j-1][0].set_ydata(-np.array([pos[j, 1], pos[j_parent, 1]]))
                    lines_3d[n][j-1][0].set_3d_properties(np.array([pos[j, 2], pos[j_parent, 2]]), zdir='y')
                else:
                    lines_3d[n][j-1][0].set_color("white")
                    lines_3d[n][j-1][0].set_xdata(np.array([0, 0]))
                    lines_3d[n][j-1][0].set_ydata(np.array([0, 0]))
                    lines_3d[n][j-1][0].set_3d_properties(np.array([0, 0]), zdir='y')
            
            # ADDED BONES
            if keypoints[i, 11, 2] != 0 and keypoints[i, 12, 2] != 0:
                lines[len(lines)-2][0].set_color('pink')
                extra_parent = 12
                lines[len(lines)-2][0].set_data([keypoints[i, 11, 0], keypoints[i, extra_parent, 0]],
                                            [keypoints[i, 11, 1], keypoints[i, extra_parent, 1]])
            if  keypoints[i, 5, 2] != 0 and keypoints[i, 6, 2] != 0:
                lines[len(lines)-1][0].set_color('pink')
                extra_parent = 6
                lines[len(lines)-1][0].set_data([keypoints[i, 5, 0], keypoints[i, extra_parent, 0]],
                                            [keypoints[i, 5, 1], keypoints[i, extra_parent, 1]])
                        
            if keypoints[i, 11, 2] != 0 and keypoints[i, 12, 2] != 0:
                extra_parent = 12
                lines_3d[n][len(lines_3d[n])-2][0].set_color("blue")
                lines_3d[n][len(lines_3d[n])-2][0].set_xdata(np.array([pos[11, 0], pos[extra_parent, 0]]))
                lines_3d[n][len(lines_3d[n])-2][0].set_ydata(-np.array([pos[11, 1], pos[extra_parent, 1]]))
                lines_3d[n][len(lines_3d[n])-2][0].set_3d_properties(np.array([pos[11, 2], pos[extra_parent, 2]]), zdir='y')
                        
            if  keypoints[i, 5, 2] != 0 and keypoints[i, 6, 2] != 0:
                extra_parent = 6
                lines_3d[n][len(lines_3d[n])-1][0].set_color("blue")
                lines_3d[n][len(lines_3d[n])-1][0].set_xdata(np.array([pos[5, 0], pos[extra_parent, 0]]))
                lines_3d[n][len(lines_3d[n])-1][0].set_ydata(-np.array([pos[5, 1], pos[extra_parent, 1]]))
                lines_3d[n][len(lines_3d[n])-1][0].set_3d_properties(np.array([pos[5, 2], pos[extra_parent, 2]]), zdir='y')
                      
            points.set_offsets(keypoints[i, :, :2])
            
        
        print('{}/{}      '.format(i, limit), end='\r')
        

    fig.tight_layout()
    print("Limit: ", limit)
    anim = FuncAnimation(fig, update_video, frames=np.arange(0, limit-1), interval=1000/fps, repeat=False)
    if output.endswith('.mp4'):
        from IPython.display import HTML
        HTML(anim.to_html5_video())  
        Writer = writers['ffmpeg']
        writer = Writer(fps=fps, metadata={}, bitrate=bitrate)
        plt.show()
        anim.save(output, writer=writer)
    elif output.endswith('.gif'):
        plt.show()
        anim.save(output, dpi=80, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')
    plt.close()
    
    

### Visualize the groundtruth

In [5]:
START_FRAME = 0
N_FRAMES = 200

In [6]:
# Consider dataset
dataset = read_json("vp1_coco_format.json")
npz_2d_dataset = read_json("data_2d_dad_gt_train.npz")
npz_3d_dataset = read_json("data_3d_dad_train.npz")

In [7]:
# images
images_list =[dataset["images"][i]["file_name"] for i in range(START_FRAME, START_FRAME + N_FRAMES)]
print(len(images_list))

# keypoints 2d
keypoints_2d = np.array([[dataset["annotations"][i]["keypoints"]] for i in range(START_FRAME, START_FRAME + N_FRAMES)])
keypoints_2d = np.reshape(keypoints_2d, (N_FRAMES, 17, 3))
print(keypoints_2d.shape)

# keypoints 3d
poses_3d = np.array([[dataset["annotations"][i]["keypoints_3d"]] for i in range(START_FRAME, START_FRAME + N_FRAMES)])
poses_3d = np.reshape(poses_3d, (1, N_FRAMES, 17, 3))

skeleton = Skeleton(parents=[-1, 0, 0, 1, 2, 0, 0, 5, 6, 7, 8, 5, 6, 11, 12, 13, 14], 
                    joints_left=[1, 3, 5, 7, 9, 11, 13, 15], 
                    joints_right=[2, 4, 6, 8, 10, 12, 14, 16])

200
(200, 17, 3)


In [14]:
render_animation(keypoints_2d, poses_3d, images_list, skeleton, 
                     output='rendering/vp1_visu_test_azim_250.mp4',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=1, size=6, azim=250, bitrate=-1, limit=200)

frames length:  (200, 1024, 1280, 3) 200 (1024, 1280, 3)

Limit:  200


/tmp/ipykernel_37780/3377187794.py:221: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [ ]:

def render_image(keypoints, poses_3d, images_list, skeleton, 
                     output='output.png',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=3, size=6, azim=180, i=100, elev=15., title="Json"):
    
    plt.ioff()
    fig = plt.figure(figsize=(size*(1 + len(poses_3d)), size))
    ax_in = fig.add_subplot(1, 1 + len(poses_3d), 1)
    ax_in.get_xaxis().set_visible(False)
    ax_in.get_yaxis().set_visible(False)
    ax_in.set_axis_off()
    ax_in.set_title('Input')

    ax_3d = []
    lines_3d = []
    trajectories = []
    all_frames = images_list
    radius = 1.7
    # for index, (data) in enumerate(poses_3d[0]): #.items()
    ax = fig.add_subplot(1, 1 + len(poses_3d), 2, projection='3d')
    ax.view_init(elev=elev, azim=azim)
    # ax.set_xlim3d([-radius/2, radius/2])
    ax.set_ylim3d([0, radius])
    ax.set_xlim3d([0, radius])
    ax.set_zlim3d([-radius/2, radius/2])
    try:
        ax.set_aspect('equal')
    except NotImplementedError:
        ax.set_aspect('auto')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_zticklabels([])
    ax.dist = 7.5
    ax.set_title("Ground truth") #, pad=35
    ax_3d.append(ax)
    lines_3d.append([])
   
        
    poses_3d = list(poses_3d)
    print("frames length: ", np.array(all_frames).shape, len(all_frames),)
        
        
    initialized = False
    image = None
    lines = []
    points = None
    

        
    parents = skeleton.parents()
   
        
    joints_right_2d = [2, 4, 6, 8, 10, 12, 14, 16]
    colors_2d = np.full(keypoints.shape[1], 'black')
    colors_2d[joints_right_2d] = 'red'

    image = ax_in.imshow(plt.imread(os.path.join(datapath, all_frames[i])), aspect='equal')
            
    for j, j_parent in enumerate(parents):
        if j_parent == -1:
            continue
                    
        if len(parents) == keypoints.shape[1]:
            # Draw skeleton only if keypoints match (otherwise we don't have the parents definition)
            if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
                ax_in.plot([keypoints[i, j, 0], keypoints[i, j_parent, 0]],
                                        [keypoints[i, j, 1], keypoints[i, j_parent, 1]], c='pink')
                if j == 11 and keypoints[i, 12, 2] != 0:
                    extra_parent = 12
                    ax_in.plot([keypoints[i, j, 0], keypoints[i, extra_parent, 0]],
                                            [keypoints[i, j, 1], keypoints[i, extra_parent, 1]],  c='deeppink')
                if j == 5 and keypoints[i, 6, 2] != 0:
                    extra_parent = 6
                    ax_in.plot([keypoints[i, j, 0], keypoints[i, extra_parent, 0]],
                                            [keypoints[i, j, 1], keypoints[i, extra_parent, 1]], c='deeppink')
          
            else:
                ax_in.plot([0, 0], [0, 0], color='blue')
        col = 'red' if j in skeleton.joints_right() else 'black'
        # for n, ax in enumerate(ax_3d):
        n = 0
        pos = poses_3d[n][i]
        if keypoints[i, j, 2] != 0 and keypoints[i, j_parent, 2] != 0 :
            ax.plot([-pos[j, 0], -pos[j_parent, 0]],[-pos[j, 1], -pos[j_parent, 1]],[-pos[j, 2], -pos[j_parent, 2]], zdir='y', c=col)
            if j==11 and keypoints[i, 12, 2] != 0:
                extra_parent = 12
                ax.plot([-pos[j, 0], -pos[extra_parent, 0]],[-pos[j, 1], -pos[extra_parent, 1]],[-pos[j, 2], -pos[extra_parent, 2]], zdir='y', c="deeppink")
            if j==5 and keypoints[i, 6, 2] != 0:
                extra_parent = 6
                ax.plot([-pos[j, 0], -pos[extra_parent, 0]],[-pos[j, 1], -pos[extra_parent, 1]],[-pos[j, 2], -pos[extra_parent, 2]], zdir='y', c="deeppink")
        else:
            ax.plot([0, 0], [0, 0], [0, 0], zdir='z', c="white")

        # points = ax_in.scatter(*keypoints[i, :2].T, c='red', s=10,  edgecolors='white', zorder=10) # replaced colors_2d

    # fig.tight_layout()
    plt.title(title)
    plt.savefig('rendering/{}_image_{}.png'.format(title, i))
    # plt.imshow(plt.imread("rendering/image_{}.png"), aspect="auto")
    fig.show()   

In [38]:
npz_2d_dataset = read_json("data_2d_dad_gt_train.npz")
npz_3d_dataset = read_json("data_3d_dad_train.npz")

print()
npz_2d_dataset = npz_2d_dataset["positions_2d"].item()
npz_3d_dataset = npz_3d_dataset['positions_3d'].item()

print(type(npz_2d_dataset))
npz_2d_dataset = npz_2d_dataset["vp1"]['inner_mirror/vp1/run2_2018-05-29-14-33-44.ids_1']
print(npz_2d_dataset.shape)

npz_3d_dataset = npz_3d_dataset["vp1"]['inner_mirror/vp1/run2_2018-05-29-14-33-44.ids_1']

npz_2d_dataset = np.reshape(npz_2d_dataset, (len(npz_2d_dataset), 17, 3))
npz_3d_dataset = np.reshape(npz_3d_dataset, (1, len(npz_3d_dataset), 17, 3))
npz_2d_dataset = np.asarray(npz_2d_dataset, dtype=np.float64)
npz_3d_dataset = np.asarray(npz_3d_dataset, dtype=np.float64)



<class 'dict'>
(42126, 51)


In [20]:
render_animation(npz_2d_dataset, npz_3d_dataset, images_list, skeleton, 
                     output='rendering/NPZ_visu_test_azim_250.mp4',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=1, size=6, azim=250, bitrate=-1, limit=200)

frames length:  (200, 1024, 1280, 3) 200 (1024, 1280, 3)

Limit:  200


/tmp/ipykernel_37780/3377187794.py:221: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [33]:
a = np.random.random((4, 17, 2))
print(a.shape)
b = np.ones((a.shape[0],a.shape[1],a.shape[2]+1)); 
b[:,:,:-1] = a
print(b.shape)
print("B shape")
print(b)

(4, 17, 2)
(4, 17, 3)
B shape
[[[4.98097203e-01 8.37166122e-01 1.00000000e+00]
  [2.35052729e-01 8.37509987e-01 1.00000000e+00]
  [4.63349316e-01 3.22501867e-01 1.00000000e+00]
  [1.88594002e-01 2.53902835e-02 1.00000000e+00]
  [1.04923169e-01 5.48464896e-01 1.00000000e+00]
  [1.80283771e-01 2.51391124e-01 1.00000000e+00]
  [7.42479592e-02 2.75459540e-01 1.00000000e+00]
  [7.41957881e-01 3.50805237e-01 1.00000000e+00]
  [1.65611980e-01 5.77418652e-01 1.00000000e+00]
  [6.35857557e-01 2.57727950e-01 1.00000000e+00]
  [8.28338665e-01 2.40711215e-01 1.00000000e+00]
  [3.76473594e-04 9.18833847e-01 1.00000000e+00]
  [7.39872233e-01 2.92345451e-01 1.00000000e+00]
  [2.12150094e-01 5.15867404e-01 1.00000000e+00]
  [2.07971961e-01 7.20139890e-01 1.00000000e+00]
  [1.93362532e-01 6.35813679e-05 1.00000000e+00]
  [4.38384426e-01 1.01677267e-01 1.00000000e+00]]

 [[6.28484714e-01 6.20601475e-01 1.00000000e+00]
  [8.97505636e-01 6.94838231e-01 1.00000000e+00]
  [8.52576304e-01 8.21786538e-01 1.00

In [39]:
from common.camera import *
X = npz_2d_dataset[..., :2].copy()
npz_2d_dataset_trsf =  normalize_screen_coordinates(X,  w=1280, h=1024)
npz_2d_dataset_trsf = image_coordinates(npz_2d_dataset_trsf, w=1280, h=1024)

X shape:  (42126, 17, 2)


In [40]:
b = np.ones((npz_2d_dataset_trsf.shape[0],npz_2d_dataset_trsf.shape[1],npz_2d_dataset_trsf.shape[2]+1)); 
b[:,:,:-1] = npz_2d_dataset_trsf
print(b.shape)
b.dtype

(42126, 17, 3)


dtype('float64')

In [41]:
render_animation(npz_2d_dataset, npz_3d_dataset, images_list, skeleton, 
                     output='rendering/NPZ_visu_test_azim_correct_input.mp4',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=1, size=6, azim=250, bitrate=-1, limit=200)

frames length:  (200, 1024, 1280, 3) 200 (1024, 1280, 3)

Limit:  200


/tmp/ipykernel_37780/3377187794.py:221: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()


In [43]:
for i in range(len(b)):
    for j in range(17):
        if b[i, j, 0] == 0 and  b[i, j, 1] == 0:
            b[i, j, 2] = 0

In [44]:
render_animation(b, npz_3d_dataset, images_list, skeleton, 
                     output='rendering/NPZ_visu_test_azim_unormalized_input.mp4',
                     datapath='/datasets_local/DriveAndAct', 
                     skip=0, fps=30, downsample=1, size=6, azim=250, bitrate=-1, limit=200)

frames length:  (200, 1024, 1280, 3) 200 (1024, 1280, 3)

Limit:  200


/tmp/ipykernel_37780/3377187794.py:221: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
